In [1]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast
import glob
import pandas_profiling as pp
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr, LogisticRegressionCV as lrcv

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
gbg = pb.retrosheet.season_game_logs(2008)

In [7]:
seasons = []
for x in range(2008,2020,1):
    seasons.append(x)

In [8]:
seasons.pop()

2019

In [9]:
seasons

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [17]:
def season_pull(year):
    name = 'overall_'+str(year)
    '''Pull from retrosheet for season indicated by year 
    EX: season_pull(2008) returns a DataFrame containing 
    the entire game by game for the 2008 MLB Season'''
    df=pb.retrosheet.season_game_logs(year)
    df.to_csv('C:/Users/dakot/Documents/GitHub/baseball_predict/Data/'+name+'.csv' )
    return df

In [18]:
for x in seasons:
    season_pull(x)

In [19]:
df = pd.read_csv('C:/Users/dakot/Documents/GitHub/baseball_predict/Data/overall_2008.csv')
cols = []
for x in df.columns:
    cols.append(x)

In [20]:
cols.pop(0)

'Unnamed: 0'

In [21]:
path = 'baseball_predict/Data/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename,usecols=cols,index_col=None)
    li.append(df)

frame = pd.concat(li, axis=0)

In [22]:
frame.shape

(26726, 161)

In [23]:
frame.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,visiting_score,home_score,num_outs,day_night,completion_info,forfeit_info,protest_info,park_id,attendance,time_of_game_minutes,visiting_line_score,home_line_score,visiting_abs,visiting_hits,visiting_doubles,visiting_triples,visiting_homeruns,visiting_rbi,visiting_sac_hits,visiting_sac_flies,visiting_hbp,visiting_bb,visiting_iw,visiting_k,visiting_sb,visiting_cs,visiting_gdp,visiting_ci,visiting_lob,visiting_pitchers_used,visiting_individual_er,visiting_er,visiting__wp,visiting_balks,visiting_po,visiting_assists,visiting_errors,visiting_pb,visiting_dp,visiting_tp,home_abs,home_hits,home_doubles,home_triples,home_homeruns,home_rbi,home_sac_hits,home_sac_flies,home_hbp,home_bb,home_iw,home_k,home_sb,home_cs,home_gdp,home_ci,home_lob,home_pitchers_used,home_individual_er,home_er,home_wp,home_balks,home_po,home_assists,home_errors,home_pb,home_dp,home_tp,ump_home_id,ump_home_name,ump_first_id,ump_first_name,ump_second_id,ump_second_name,ump_third_id,ump_third_name,ump_lf_id,ump_lf_name,ump_rf_id,ump_rf_name,visiting_manager_id,visiting_manager_name,home_manager_id,home_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,save_pitcher_id,save_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,visiting_starting_pitcher_id,visiting_starting_pitcher_name,home_starting_pitcher_id,home_starting_pitcher_name,visiting_1_id,visiting_1_name,visiting_1_pos,visiting_2_id,visiting_2_name,visiting_2_pos,visiting_2_id.1,visiting_3_name,visiting_3_pos,visiting_4_id,visiting_4_name,visiting_4_pos,visiting_5_id,visiting_5_name,visiting_5_pos,visiting_6_id,visiting_6_name,visiting_6_pos,visiting_7_id,visiting_7_name,visiting_7_pos,visiting_8_id,visiting_8_name,visiting_8_pos,visiting_9_id,visiting_9_name,visiting_9_pos,home_1_id,home_1_name,home_1_pos,home_2_id,home_2_name,home_2_pos,home_3_id,home_3_name,home_3_pos,home_4_id,home_4_name,home_4_pos,home_5_id,home_5_name,home_5_pos,home_6_id,home_6_name,home_6_pos,home_7_id,home_7_name,home_7_pos,home_8_id,home_8_name,home_8_pos,home_9_id,home_9_name,home_9_pos,misc,acquisition_info
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,6,5,60,N,NaN,NaN,NaN,TOK01,44628.0,219,0000030012,2000020001,38,11,3,0,1,6,1,0,0,3,2,7,0,0,2,0,6,6,5,5,1,0,30,15,0,0,0,0,36,7,1,0,2,5,0,0,1,7,0,10,1,0,0,0,9,5,6,6,0,0,30,12,0,0,2,0,reedr901,Rick Reed,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,okajh001,Hideki Okajima,streh001,Huston Street,papej001,Jonathan Papelbon,ramim002,Manny Ramirez,matsd001,Daisuke Matsuzaka,blanj001,Joe Blanton,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,ellsj001,Jacoby Ellsbury,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,custj001,Jack Cust,10,browe001,Emil Brown,7,crosb002,Bobby Crosby,6,hannj001,Jack Hannahan,5,suzuk001,Kurt Suzuki,2,sweer001,Ryan Sweeney,8,NaN,Y
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,1,5,51,N,NaN,NaN,NaN,TOK01,44735.0,171,000001000,01300001x,31,5,1,0,1,1,0,0,0,3,0,13,1,0,2,0,6,5,5,5,0,0,24,8,0,0,0,0,33,9,3,0,1,5,0,0,0,4,0,7,0,0,0,0,8,4,1,1,1,0,27,5,1,0,2,0,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,reedr901,Rick Reed,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,hardr001,Rich Harden,lestj001,Jon Lester,NaN,(none),denoc001,Chris Denorfia,lestj001,Jon Lester,hardr001,Rich Harden,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,crisc001,Coco Crisp,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,sweem002,Mike Sweeney,10,browe001,Emil Brown,7,crosb002,Bobby Crosby

In [24]:
for x in frame:
    frame['win'] = np.where(frame['home_score']>frame['visiting_score'],frame['home_team'],frame['visiting_team'])

In [25]:
frame.win.describe()

count     26726
unique       31
top         NYA
freq       1010
Name: win, dtype: object

In [26]:
frame.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,visiting_score,home_score,num_outs,day_night,completion_info,forfeit_info,protest_info,park_id,attendance,time_of_game_minutes,visiting_line_score,home_line_score,visiting_abs,visiting_hits,visiting_doubles,visiting_triples,visiting_homeruns,visiting_rbi,visiting_sac_hits,visiting_sac_flies,visiting_hbp,visiting_bb,visiting_iw,visiting_k,visiting_sb,visiting_cs,visiting_gdp,visiting_ci,visiting_lob,visiting_pitchers_used,visiting_individual_er,visiting_er,visiting__wp,visiting_balks,visiting_po,visiting_assists,visiting_errors,visiting_pb,visiting_dp,visiting_tp,home_abs,home_hits,home_doubles,home_triples,home_homeruns,home_rbi,home_sac_hits,home_sac_flies,home_hbp,home_bb,home_iw,home_k,home_sb,home_cs,home_gdp,home_ci,home_lob,home_pitchers_used,home_individual_er,home_er,home_wp,home_balks,home_po,home_assists,home_errors,home_pb,home_dp,home_tp,ump_home_id,ump_home_name,ump_first_id,ump_first_name,ump_second_id,ump_second_name,ump_third_id,ump_third_name,ump_lf_id,ump_lf_name,ump_rf_id,ump_rf_name,visiting_manager_id,visiting_manager_name,home_manager_id,home_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,save_pitcher_id,save_pitcher_name,game_winning_rbi_id,game_winning_rbi_name,visiting_starting_pitcher_id,visiting_starting_pitcher_name,home_starting_pitcher_id,home_starting_pitcher_name,visiting_1_id,visiting_1_name,visiting_1_pos,visiting_2_id,visiting_2_name,visiting_2_pos,visiting_2_id.1,visiting_3_name,visiting_3_pos,visiting_4_id,visiting_4_name,visiting_4_pos,visiting_5_id,visiting_5_name,visiting_5_pos,visiting_6_id,visiting_6_name,visiting_6_pos,visiting_7_id,visiting_7_name,visiting_7_pos,visiting_8_id,visiting_8_name,visiting_8_pos,visiting_9_id,visiting_9_name,visiting_9_pos,home_1_id,home_1_name,home_1_pos,home_2_id,home_2_name,home_2_pos,home_3_id,home_3_name,home_3_pos,home_4_id,home_4_name,home_4_pos,home_5_id,home_5_name,home_5_pos,home_6_id,home_6_name,home_6_pos,home_7_id,home_7_name,home_7_pos,home_8_id,home_8_name,home_8_pos,home_9_id,home_9_name,home_9_pos,misc,acquisition_info,win
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,6,5,60,N,NaN,NaN,NaN,TOK01,44628.0,219,0000030012,2000020001,38,11,3,0,1,6,1,0,0,3,2,7,0,0,2,0,6,6,5,5,1,0,30,15,0,0,0,0,36,7,1,0,2,5,0,0,1,7,0,10,1,0,0,0,9,5,6,6,0,0,30,12,0,0,2,0,reedr901,Rick Reed,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,okajh001,Hideki Okajima,streh001,Huston Street,papej001,Jonathan Papelbon,ramim002,Manny Ramirez,matsd001,Daisuke Matsuzaka,blanj001,Joe Blanton,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,ellsj001,Jacoby Ellsbury,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,custj001,Jack Cust,10,browe001,Emil Brown,7,crosb002,Bobby Crosby,6,hannj001,Jack Hannahan,5,suzuk001,Kurt Suzuki,2,sweer001,Ryan Sweeney,8,NaN,Y,BOS
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,1,5,51,N,NaN,NaN,NaN,TOK01,44735.0,171,000001000,01300001x,31,5,1,0,1,1,0,0,0,3,0,13,1,0,2,0,6,5,5,5,0,0,24,8,0,0,0,0,33,9,3,0,1,5,0,0,0,4,0,7,0,0,0,0,8,4,1,1,1,0,27,5,1,0,2,0,wendh902,Hunter Wendelstedt,timmt901,Tim Timmons,nauep901,Paul Nauert,reedr901,Rick Reed,NaN,(none),NaN,(none),frant001,Terry Francona,gereb001,Bob Geren,hardr001,Rich Harden,lestj001,Jon Lester,NaN,(none),denoc001,Chris Denorfia,lestj001,Jon Lester,hardr001,Rich Harden,pedrd001,Dustin Pedroia,4,youkk001,Kevin Youkilis,3,ortid001,David Ortiz,10,ramim002,Manny Ramirez,7,lowem001,Mike Lowell,5,mossb001,Brandon Moss,9,varij001,Jason Varitek,2,crisc001,Coco Crisp,8,lugoj001,Julio Lugo,6,buckt001,Travis Buck,9,ellim001,Mark Ellis,4,bartd001,Daric Barton,3,sweem002,Mike Sweeney,10,browe001,Emil Brown,7,crosb002,Bobb

In [27]:
for x in frame.columns:
    print(x)

date
game_num
day_of_week
visiting_team
visiting_team_league
visiting_game_num
home_team
home_team_league
home_team_game_num
visiting_score
home_score
num_outs
day_night
completion_info
forfeit_info
protest_info
park_id
attendance
time_of_game_minutes
visiting_line_score
home_line_score
visiting_abs
visiting_hits
visiting_doubles
visiting_triples
visiting_homeruns
visiting_rbi
visiting_sac_hits
visiting_sac_flies
visiting_hbp
visiting_bb
visiting_iw
visiting_k
visiting_sb
visiting_cs
visiting_gdp
visiting_ci
visiting_lob
visiting_pitchers_used
visiting_individual_er
visiting_er
visiting__wp
visiting_balks
visiting_po
visiting_assists
visiting_errors
visiting_pb
visiting_dp
visiting_tp
home_abs
home_hits
home_doubles
home_triples
home_homeruns
home_rbi
home_sac_hits
home_sac_flies
home_hbp
home_bb
home_iw
home_k
home_sb
home_cs
home_gdp
home_ci
home_lob
home_pitchers_used
home_individual_er
home_er
home_wp
home_balks
home_po
home_assists
home_errors
home_pb
home_dp
home_tp
ump_home_id
u

In [28]:
#isolate all columns with future information, these details wouldnt be known until the conclusion of the game 
drops = ['visiting_score',
'home_score',
'num_outs',
'completion_info',
'forfeit_info',
'protest_info',
'attendance',
'time_of_game_minutes',
'visiting_line_score',
'home_line_score',
'visiting_abs',
'visiting_hits',
'visiting_doubles',
'visiting_triples',
'visiting_homeruns',
'visiting_rbi',
'visiting_sac_hits',
'visiting_sac_flies',
'visiting_hbp',
'visiting_bb',
'visiting_iw',
'visiting_k',
'visiting_sb',
'visiting_cs',
'visiting_gdp',
'visiting_ci',
'visiting_lob',
'visiting_pitchers_used',
'visiting_individual_er',
'visiting_er',
'visiting__wp',
'visiting_balks',
'visiting_po',
'visiting_assists',
'visiting_errors',
'visiting_pb',
'visiting_dp',
'visiting_tp',
'home_abs',
'home_hits',
'home_doubles',
'home_triples',
'home_homeruns',
'home_rbi',
'home_sac_hits',
'home_sac_flies',
'home_hbp',
'home_bb',
'home_iw',
'home_k',
'home_sb',
'home_cs',
'home_gdp',
'home_ci',
'home_lob',
'home_pitchers_used',
'home_individual_er',
'home_er',
'home_wp',
'home_balks',
'home_po',
'home_assists',
'home_errors',
'home_pb',
'home_dp',
'home_tp',
'winning_pitcher_id',
'winning_pitcher_name',
'losing_pitcher_id',
'losing_pitcher_name',
'save_pitcher_id',
'save_pitcher_name',
'game_winning_rbi_id',
'game_winning_rbi_name',
'misc',
'acquisition_info',
'visiting_manager_id',
'home_manager_id',
'visiting_starting_pitcher_id',
'home_starting_pitcher_id',
'visiting_1_id',
'visiting_2_id',
'visiting_2_id.1',
'visiting_4_id',
'visiting_5_id',
'visiting_6_id',
'visiting_7_id',
'visiting_8_id',
'visiting_9_id',
'home_1_id',
'home_2_id',
'home_3_id',
'home_4_id',
'home_5_id',
'home_6_id',
'home_7_id',
'home_8_id',
'home_9_id',
'ump_home_id',
'ump_home_name',
'ump_first_id',
'ump_first_name',
'ump_second_id',
'ump_second_name',
'ump_third_id',
'ump_third_name' ,
'ump_lf_id',
'ump_lf_name',
'ump_rf_id',
'ump_rf_name']

In [29]:
kill_time_trvl = frame.drop(drops, axis=1)

In [30]:
kill_time_trvl.head()

,date,game_num,day_of_week,visiting_team,visiting_team_league,visiting_game_num,home_team,home_team_league,home_team_game_num,day_night,park_id,visiting_manager_name,home_manager_name,visiting_starting_pitcher_name,home_starting_pitcher_name,visiting_1_name,visiting_1_pos,visiting_2_name,visiting_2_pos,visiting_3_name,visiting_3_pos,visiting_4_name,visiting_4_pos,visiting_5_name,visiting_5_pos,visiting_6_name,visiting_6_pos,visiting_7_name,visiting_7_pos,visiting_8_name,visiting_8_pos,visiting_9_name,visiting_9_pos,home_1_name,home_1_pos,home_2_name,home_2_pos,home_3_name,home_3_pos,home_4_name,home_4_pos,home_5_name,home_5_pos,home_6_name,home_6_pos,home_7_name,home_7_pos,home_8_name,home_8_pos,home_9_name,home_9_pos,win
0,20080325,0,Tue,BOS,AL,1,OAK,AL,1,N,TOK01,Terry Francona,Bob Geren,Daisuke Matsuzaka,Joe Blanton,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Jacoby Ellsbury,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Jack Cust,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Ryan Sweeney,8,BOS
1,20080326,0,Wed,BOS,AL,2,OAK,AL,2,N,TOK01,Terry Francona,Bob Geren,Jon Lester,Rich Harden,Dustin Pedroia,4,Kevin Youkilis,3,David Ortiz,10,Manny Ramirez,7,Mike Lowell,5,Brandon Moss,9,Jason Varitek,2,Coco Crisp,8,Julio Lugo,6,Travis Buck,9,Mark Ellis,4,Daric Barton,3,Mike Sweeney,10,Emil Brown,7,Bobby Crosby,6,Jack Hannahan,5,Kurt Suzuki,2,Chris Denorfia,8,OAK
2,20080330,0,Sun,ATL,NL,1,WAS,NL,1,N,WAS11,Bobby Cox,Manny Acta,Tim Hudson,Odalis Perez,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Jeff Francoeur,9,Brian McCann,2,Matt Diaz,7,Mark Kotsay,8,Tim Hudson,1,Cristian Guzman,6,Lastings Milledge,8,Ryan Zimmerman,5,Nick Johnson,3,Austin Kearns,9,Paul Lo Duca,2,Elijah Dukes,7,Ronnie Belliard,4,Odalis Perez,1,WAS
3,20080331,0,Mon,PIT,NL,1,ATL,NL,2,N,ATL02,John Russell,Bobby Cox,Ian Snell,Tom Glavine,Nate McLouth,8,Freddy Sanchez,4,Jason Bay,7,Adam LaRoche,3,Xavier Nady,9,Ryan Doumit,2,Jose Bautista,5,Jack Wilson,6,Ian Snell,1,Kelly Johnson,4,Yunel Escobar,6,Chipper Jones,5,Mark Teixeira,3,Brian McCann,2,Jeff Francoeur,9,Mark Kotsay,8,Matt Diaz,7,Tom Glavine,1,PIT
4,20080331,0,Mon,MIL,NL,1,CHN,NL,1,D,CHI11,Ned Yost,Lou Piniella,Ben Sheets,Carlos Zambrano,Rickie Weeks,4,Tony Gwynn,8,Prince Fielder,3,Ryan Braun,7,Bill Hall,5,Corey Hart,9,J.J. Hardy,6,Ben Sheets,1,Jason Kendall,2,Ryan Theriot,6,Alfonso Soriano,7,Derrek Lee,3,Aramis Ramirez,5,Kosuke Fukudome,9,Mark DeRosa,4,Geovany Soto,2,Felix Pie,8,Carlos Zambrano,1,MIL


In [31]:
kill_time_trvl.describe(include='object')

,day_of_week,visiting_team,visiting_team_league,home_team,home_team_league,day_night,park_id,visiting_manager_name,home_manager_name,visiting_starting_pitcher_name,home_starting_pitcher_name,visiting_1_name,visiting_2_name,visiting_3_name,visiting_4_name,visiting_5_name,visiting_6_name,visiting_7_name,visiting_8_name,visiting_9_name,home_1_name,home_2_name,home_3_name,home_4_name,home_5_name,home_6_name,home_7_name,home_8_name,home_9_name,win
count,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726,26726
unique,7,31,2,31,2,2,42,97,99,961,953,612,881,596,598,921,1167,1334,1488,1790,602,891,581,588,896,1155,1328,1430,1381,31
top,Sat,MIN,NL,LAN,NL,N,MIL06,Joe Maddon,Joe Maddon,Justin Verlander,James Shields,Denard Span,Dustin Pedroia,Joey Votto,Adrian Beltre,Jay Bruce,Yadier Molina,Mark Reynolds,Jeff Mathis,Cliff Pennington,Denard Span,Dustin Pedroia,Ryan Braun,Adrian Beltre,Jay Bruce,Yadier Molina,J.J. Hardy,Jordy Mercer,Brendan Ryan,NYA
freq,4337,892,13766,892,13766,17947,896,892,892,179,182,585,408,639,523,319,235,154,213,196,572,416,622,522,299,256,157,225,200,1010


In [32]:
kill_time_trvl.isna().sum()

date                              0
game_num                          0
day_of_week                       0
visiting_team                     0
visiting_team_league              0
visiting_game_num                 0
home_team                         0
home_team_league                  0
home_team_game_num                0
day_night                         0
park_id                           0
visiting_manager_name             0
home_manager_name                 0
visiting_starting_pitcher_name    0
home_starting_pitcher_name        0
visiting_1_name                   0
visiting_1_pos                    0
visiting_2_name                   0
visiting_2_pos                    0
visiting_3_name                   0
visiting_3_pos                    0
visiting_4_name                   0
visiting_4_pos                    0
visiting_5_name                   0
visiting_5_pos                    0
visiting_6_name                   0
visiting_6_pos                    0
visiting_7_name             

In [33]:
kill_time_trvl.to_csv('C:\\Users\\dakot\\Documents\\GitHub\\baseball_predict\\Data\\kill_time_trvl.csv')

In [34]:
y = kill_time_trvl['win']
X = kill_time_trvl.drop(columns='win', axis = 1)

In [35]:
y.shape


(26726,)

In [36]:
# Need to encode X to model logistic, multiclass 
# Look into wrapping multiple paired team models in a single function 
# Deploy the thing!

In [37]:
X.columns

Index(['date', 'game_num', 'day_of_week', 'visiting_team', 'visiting_team_league', 'visiting_game_num', 'home_team', 'home_team_league', 'home_team_game_num', 'day_night', 'park_id', 'visiting_manager_name', 'home_manager_name', 'visiting_starting_pitcher_name', 'home_starting_pitcher_name', 'visiting_1_name', 'visiting_1_pos', 'visiting_2_name', 'visiting_2_pos', 'visiting_3_name', 'visiting_3_pos', 'visiting_4_name', 'visiting_4_pos', 'visiting_5_name', 'visiting_5_pos', 'visiting_6_name', 'visiting_6_pos', 'visiting_7_name', 'visiting_7_pos', 'visiting_8_name', 'visiting_8_pos', 'visiting_9_name', 'visiting_9_pos', 'home_1_name', 'home_1_pos', 'home_2_name', 'home_2_pos', 'home_3_name', 'home_3_pos', 'home_4_name', 'home_4_pos', 'home_5_name', 'home_5_pos', 'home_6_name', 'home_6_pos', 'home_7_name', 'home_7_pos', 'home_8_name', 'home_8_pos', 'home_9_name', 'home_9_pos'], dtype='object')

In [38]:
# use Binary encoding to take care of categoricals and labels without adding too much dimensionality 
import category_encoders as ce

In [39]:
# Categorical boolean mask
cat_mask = X.dtypes==object
# filter categorical columns using mask and turn it into a list
cat_cols = X.columns[cat_mask].tolist()

In [40]:
ids = ['visiting_manager_id',
'home_manager_id',
'visiting_starting_pitcher_id',
'home_starting_pitcher_id',
'visiting_1_id',
'visiting_2_id',
'visiting_2_id.1',
'visiting_4_id',
'visiting_5_id',
'visiting_6_id',
'visiting_7_id',
'visiting_8_id',
'visiting_9_id',
'home_1_id',
'home_2_id',
'home_3_id',
'home_4_id',
'home_5_id',
'home_6_id',
'home_7_id',
'home_8_id',
'home_9_id']

In [41]:
for x in cat_cols:
    for i in ids:
        if x==i:
            cat_cols.remove(x)

In [42]:
cat_cols


['day_of_week',
 'visiting_team',
 'visiting_team_league',
 'home_team',
 'home_team_league',
 'day_night',
 'park_id',
 'visiting_manager_name',
 'home_manager_name',
 'visiting_starting_pitcher_name',
 'home_starting_pitcher_name',
 'visiting_1_name',
 'visiting_2_name',
 'visiting_3_name',
 'visiting_4_name',
 'visiting_5_name',
 'visiting_6_name',
 'visiting_7_name',
 'visiting_8_name',
 'visiting_9_name',
 'home_1_name',
 'home_2_name',
 'home_3_name',
 'home_4_name',
 'home_5_name',
 'home_6_name',
 'home_7_name',
 'home_8_name',
 'home_9_name']

In [43]:
encoder = ce.BinaryEncoder(cols=cat_cols)
X_cat_enc = encoder.fit_transform(X)

In [44]:
X_cat_enc.shape #soooo many columns, but we shall workwith it for now

(26726, 295)

In [45]:
X_cat_enc.shape, y.shape

((26726, 295), (26726,))

In [46]:
X_train, X_test, y_train, y_test = tts(X_cat_enc,y,test_size=0.2, random_state=42 )

In [47]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((21380, 295), (5346, 295), (21380,), (5346,))

In [48]:
# l_model = lr(multi_class='multinomial',solver='newton-cg', max_iter=300)
# l_model.fit(X_train,y_train)

In [49]:
# drag df back in for other tests and less overhead 
# will need to split again 
# try catboost without encoding

In [2]:
import pybaseball as pb 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pybaseball import statcast
import glob
import pandas_profiling as pp
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as lr, LogisticRegressionCV as lrcv

In [3]:
reup = pd.read_csv('C:\\Users\\dakot\\Documents\\GitHub\\baseball_predict\\Data\\kill_time_trvl.csv')
reup.shape

(26726, 53)

In [4]:
X_2 = reup.drop('win',axis=1)
y_2 = reup['win']

In [5]:
X_tr, X_tst, y_tr, y_tst = tts(X_2,y_2,test_size=0.2, random_state=42 )

In [6]:
from sklearn.metrics import accuracy_score as score

In [7]:
from catboost import CatBoostClassifier as cb


In [8]:
# X_2.dtypes

In [9]:
categorical_features_indices = np.where(X_2.dtypes != np.int64)[0]

In [10]:
categorical_features_indices

array([ 3,  4,  5,  7,  8, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 24, 26,
       28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50], dtype=int64)

In [22]:
kitty = cb(
    iterations=40,
    depth=5,
    learning_rate=0.01,
    #random_seed=42,
    loss_function='MultiClass',
    verbose=True)



In [23]:
kitty.fit(X_tr,y_tr,cat_features=categorical_features_indices)

0:	learn: 3.4022973	total: 2.47s	remaining: 1m 36s
1:	learn: 3.3717512	total: 5.05s	remaining: 1m 36s
2:	learn: 3.3395028	total: 7.75s	remaining: 1m 35s
3:	learn: 3.3073887	total: 11.5s	remaining: 1m 43s
4:	learn: 3.2851632	total: 15.5s	remaining: 1m 48s
5:	learn: 3.2558654	total: 19.2s	remaining: 1m 48s
6:	learn: 3.2272291	total: 22.9s	remaining: 1m 47s
7:	learn: 3.1995604	total: 25.7s	remaining: 1m 42s
8:	learn: 3.1757376	total: 28.5s	remaining: 1m 38s
9:	learn: 3.1510298	total: 31.1s	remaining: 1m 33s
10:	learn: 3.1303495	total: 33.6s	remaining: 1m 28s
11:	learn: 3.1095467	total: 36.2s	remaining: 1m 24s
12:	learn: 3.0878986	total: 38.6s	remaining: 1m 20s
13:	learn: 3.0696627	total: 41.2s	remaining: 1m 16s
14:	learn: 3.0467020	total: 43.8s	remaining: 1m 12s
15:	learn: 3.0304737	total: 46.3s	remaining: 1m 9s
16:	learn: 3.0120203	total: 48.8s	remaining: 1m 6s
17:	learn: 2.9929178	total: 51.4s	remaining: 1m 2s
18:	learn: 2.9773138	total: 53.9s	remaining: 59.6s
19:	learn: 2.9600435	total

In [24]:
pred_class = kitty.predict(X_tst)
pred_probs = kitty.predict_proba(X_tst)

In [25]:
cat_score = score(y_tst,pred_class)
print('Accuracy of catboost=',cat_score)

Accuracy of catboost= 0.5362888140665918


In [26]:
imps = kitty.get_feature_importance(prettified=True)
scrs = kitty.score(X_tst,y_tst)
probs = kitty.predict_proba(data=(X_tst[:1]))

In [31]:
imps
scrs
# probs

0.5362888140665918

In [ ]:
# Pickle the darn thing! 🥒

In [0]:
# explainer = shap.TreeExplainer(kitty) # WHY DOES THIS TAKE SOO LONG?!?!?!?!
# shap_values = explainer.shap_values(X_test)

In [0]:
y_pred = l_model.predict(X_test)

In [0]:
acc = score(y_test,y_pred)

In [0]:
acc

In [0]:
l_model.coef_

In [0]:
print(list(zip(l_model.coef_[0,:], X.columns)))

In [0]:
coef_dict = {}
for coef, feat in zip(l_model.coef_[0,:],X.columns):
    coef_dict[feat] = coef

In [0]:
plt.figure(figsize=(20,20))

plt.bar(*zip(*coef_dict.items()),color='orange')
plt.xticks( X.columns, rotation='vertical', fontsize='20')
plt.grid();
plt.savefig('feature_coefs.png',bbox_inches='tight')

In [0]:
pred_class